# Variogram modeling

The goal of this tutorial is to illustrate one possible workflow for fitting variograms with `GeoStats.jl`. In the future, part of this workflow will be automated and integrated with the Julia data science stack, which is gradually maturing.

Before we proceed, please install the following packages:

In [1]:
for pkg in ["GeoStats", "GeoStatsImages", "Plots", "GR", "Interact"]
    Pkg.add(pkg)
end
# make sure this tutorial is reproducible
srand(2017);

INFO: Package GeoStats is already installed
INFO: Package GeoStatsImages is already installed
INFO: Package Plots is already installed
INFO: Package GR is already installed
INFO: Package Interact is already installed


## The data

Let's consider a simple 2D problem in which properties of a field are sampled at random locations. We will be using an image of the Walker Lake in Nevada available in `GeoStatsImages.jl` as our field:

In [2]:
using GeoStatsImages
using Plots; gr()

field = training_image("WalkerLake")[:,:,1]

heatmap(field, label="Walker Lake, NV")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 
 
 
 
 
 
 
 
<image width="18" height="361" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAABIAAAFpCAYAAACRXHjhAAACEUlEQVR4nO3c0W3EMAyDYafI/lN0
y17jjpCXD8UPwR6AMElR8hkXX5/ney+wvgTIWmvdz/4YoI2AHLX9zKVWc42JPVgjR22pynZAR6O3
Jan9ICAVkaJGiNq1fwmQpGZ2dF8IKEhtIdcmazSa2mOA1I6SGvWoIdcUtdn2G7GD9rspMlejov3H
tX8EIvdQUqMd00jab8Tu1dFgjSS1GhCK2mSNitRMQugN+2WAoP1mR/IammlUo6Z2NFsjg8U0KtrP
2sg27IIdUgENruxg+ndNo2T6c6FVBenSn2tsueaftH9s+mFl96jFXDsReV9Far3KrhXk/eTmWk6j
IrUaUFCjZy61HNBgjeQfvWPUkhrFjn6D7R9MbbRGay41N/sJTnGu9Y41DyrIyRo5agSmSa3nmqps
1iGDja2WftnYENBJ/ztQMf0ICB79zCoe/QhMkprq2cmTvwLKUduM2uTGpoAG229wio1tsEaQWq2y
4ThCQL0fNWdkvwMVT2w1oMkaGZxkzz72vwJt9C1MkhoC6s21ydRcY0PfQaodyQtNRK3Y/MdSy/Xs
YPpdaHtzzYWW2V/TyFX2sf8VaKNvhYsaXT1q6MEJBSRDq6hdLCI1jZj9xdCi14agRrW3xkan/9j/
smCHRO/NBKlt9WpVr0O6iBz731bQtcmV7ajlenbNtXvn3s9m1FbuJmLVGhvbUdL+udQQ0B+p8+fh
sEs94QAAAABJRU5ErkJggg==
" transform="translate(544, 8)"/>
 
 
 0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0

We sample a thousand points from the image and plot them:

In [3]:
# sample points
nx, ny = size(field)
idxs = rand(1:nx*ny, 1000)
vals = field[idxs]

# reshape arrays
X = ind2sub((nx,ny), idxs)
X = [X[1] X[2]]'
X = Float64.(X)
z = vals

# plot (y,x) to match image orientation
scatter(X[2,:], X[1,:], c=vals)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<circle clip-path="url(#clip02)" style="fill:#000000; stroke:none; fill-opacity:1" cx="566.

## Empirical variogram

We consider a simple omnidirectional variogram. Variogram objects in GeoStats.jl are integrated with the Julia Plots.jl framework. We can plot them directly and interact with the result as we vary the number of bins used for estimation:

In [4]:
using GeoStats
using Interact

@manipulate for nbins=1:30
    γ = EmpiricalVariogram(X, z, nbins=nbins)
    
    plot(γ)
end

Interact.Options{:SelectionSlider,Int64}(1: "input" = 15 Int64 , "nbins", 15, "15", Interact.OptionDict(DataStructures.OrderedDict("1"=>1,"2"=>2,"3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10…), Dict(18=>"18",30=>"30",2=>"2",16=>"16",11=>"11",21=>"21",7=>"7",9=>"9",25=>"25",10=>"10"…)), Any[], Any[], true, "horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 Lag h 
 
 
 Variogram(h) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 bin counts 
 
 
 
 
 variogram

Besides the variogram itself, GeoStats.jl presents the bin counts (scaled) as a measure of confidence about the estimated points. This frequency plot can be deactived by passing the option `bincounts=false` to the plot command. We encourage users to keep the bin counts option activated as it has zero cost.

This empirical variogram was constructed using the Euclidean distance between data locations. We can also specify a custom distance to estimate the variogram when points are embedded on different coordinate systems. Please consult the documentation for more distance functions.

After interacting with the plot, we select a number of bins and proceed to fitting a theoretical model:

In [5]:
# final empirical variogram
γemp = EmpiricalVariogram(X, z, nbins=17)

plot(γemp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 Lag h 
 
 
 Variogram(h) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 bin counts 
 
 
 
 
 variogram

## Theoretical variogram

We will exploit the Interact.jl package again to fit a theoretical variogram to the data. Various theoretical variogram models are available in GeoStats.jl, including a composite additive model that can be used to combine different variogram types. Please consult the documentation for more details. Here we will use a simple spherical variogram:

In [6]:
@manipulate for s=linspace(0,.1), r=linspace(0,100), n=linspace(0,.1)
    # theoretical variogram
    γtheo = SphericalVariogram(sill=s, range=r, nugget=n)

    plot(γemp)
    plot!(γtheo, maxlag=500., label="theoretical")
end

Interact.Options{:SelectionSlider,Float64}(5: "input-2" = 0.04897959183673469 Float64 , "s", 0.04897959183673469, "0.04897959183673469", Interact.OptionDict(DataStructures.OrderedDict("0.0"=>0.0,"0.0020408163265306124"=>0.00204082,"0.004081632653061225"=>0.00408163,"0.006122448979591836"=>0.00612245,"0.00816326530612245"=>0.00816327,"0.01020408163265306"=>0.0102041,"0.012244897959183673"=>0.0122449,"0.014285714285714285"=>0.0142857,"0.0163265306122449"=>0.0163265,"0.018367346938775512"=>0.0183673…), Dict(0.0102041=>"0.01020408163265306",0.0183673=>"0.018367346938775512",0.0265306=>"0.026530612244897958",0.0877551=>"0.08775510204081632",0.0163265=>"0.0163265306122449",0.0693878=>"0.06938775510204082",0.0857143=>"0.08571428571428572",0.0306122=>"0.030612244897959183",0.0387755=>"0.03877551020408163",0.0367347=>"0.036734693877551024"…)), Any[], Any[], true, "horizontal")

Interact.Options{:SelectionSlider,Float64}(7: "input-3" = 48.97959183673469 Float64 , "r", 48.97959183673469, "48.97959183673469", Interact.OptionDict(DataStructures.OrderedDict("0.0"=>0.0,"2.0408163265306123"=>2.04082,"4.081632653061225"=>4.08163,"6.122448979591836"=>6.12245,"8.16326530612245"=>8.16327,"10.204081632653061"=>10.2041,"12.244897959183673"=>12.2449,"14.285714285714286"=>14.2857,"16.3265306122449"=>16.3265,"18.367346938775512"=>18.3673…), Dict(63.2653=>"63.265306122448976",75.5102=>"75.51020408163265",46.9388=>"46.93877551020408",22.449=>"22.448979591836736",55.102=>"55.10204081632653",59.1837=>"59.183673469387756",95.9184=>"95.91836734693878",93.8776=>"93.87755102040816",71.4286=>"71.42857142857143",18.3673=>"18.367346938775512"…)), Any[], Any[], true, "horizontal")

Interact.Options{:SelectionSlider,Float64}(9: "input-4" = 0.04897959183673469 Float64 , "n", 0.04897959183673469, "0.04897959183673469", Interact.OptionDict(DataStructures.OrderedDict("0.0"=>0.0,"0.0020408163265306124"=>0.00204082,"0.004081632653061225"=>0.00408163,"0.006122448979591836"=>0.00612245,"0.00816326530612245"=>0.00816327,"0.01020408163265306"=>0.0102041,"0.012244897959183673"=>0.0122449,"0.014285714285714285"=>0.0142857,"0.0163265306122449"=>0.0163265,"0.018367346938775512"=>0.0183673…), Dict(0.0102041=>"0.01020408163265306",0.0183673=>"0.018367346938775512",0.0265306=>"0.026530612244897958",0.0877551=>"0.08775510204081632",0.0163265=>"0.0163265306122449",0.0693878=>"0.06938775510204082",0.0857143=>"0.08571428571428572",0.0306122=>"0.030612244897959183",0.0387755=>"0.03877551020408163",0.0367347=>"0.036734693877551024"…)), Any[], Any[], true, "horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 Lag h 
 
 
 Variogram(h) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#008000; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,174.822 44.4314,174.822 49.4928,174.822 54.5541,174.822 59.6155,174.822 64.6768,174.822 69.7381,174.822 74.7995,174.822 79.8608,174.822 84.9222,174.822 
 89.9835,174.822 95.0448,174.822 100.106,174.822 105.168,174.822 110.229,174.822 115.29,174.822 120.352,174.822 125.413,174.822 130.474,174.822 135.536,174.822 
 140.597,174.822 145.658,174.822 150.72,174.822 155.781,174.822 160.842,174.822 165.904,174.822 170.965,174.822 176.026,174.822 181.088,174.822 186.149,174.822 
 191.21,174.822 196.272,174.822 201.333,174.822 206.394,174.822 211.456,174.822 216.517,174.822 221.578,174.822 226.64,174.822 231.701,174.822 236.762,174.822 
 241.824,174.822 246.885,174.822 251.946,174.822 257.008,174.822 262.069,174.822 267.131,174.822 272.192,174.822 277.253,174.822 282.315,174.822 287.376,174.822 
 292.437,174.822 297.499,174.822 302.56,174.822 307.621,174.822 312.683,174.822 317.744,174.822 322.805,174.822 327.867,174.822 332.928,174.822 337.989,174.822 
 343.051,174.822 348.112,174.822 353.173,174.822 358.235,174.822 363.296,174.822 368.357,174.822 373.419,174.822 378.48,174.822 383.541,174.822 388.603,174.822 
 393.664,174.822 398.725,174.822 403.787,174.822 408.848,174.822 413.909,174.822 418.971,174.822 424.032,174.822 429.093,174.822 434.155,174.822 439.216,174.822 
 444.278,174.822 449.339,174.822 454.4,174.822 459.462,174.822 464.523,174.822 469.584,174.822 474.646,174.822 479.707,174.822 484.768,174.822 489.83,174.822 
 494.891,174.822 499.952,174.822 505.014,174.822 510.075,174.822 515.136,174.822 520.198,174.822 525.259,174.822 530.32,174.822 535.382,174.822 540.443,174.822 
 
 "/>
 
 
 
 
 
 bin counts 
 
 
 
 
 variogram 
 
 
 
 theoretical

After tuning the parameters of the theoretical variogram interactively, we proceed to Kriging estimation:

In [7]:
γtheo = SphericalVariogram(sill=.083, range=32.653)

GeoStats.SphericalVariogram{Float64,GeoStats.EuclideanDistance}
  sill: Float64 0.083
  range: Float64 32.653
  nugget: Float64 0.0
  distance: GeoStats.EuclideanDistance GeoStats.EuclideanDistance()
